In [1]:
import os
import time
import zipfile
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.random_projection import SparseRandomProjection

###############################################################################
# Utility functions

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def flastVectorization(dataPoints, dim=0, eps=0.3):
    """Performs vectorization using CountVectorizer with optional dimensionality reduction."""
    countVec = CountVectorizer(stop_words=None)  # No stop word removal
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

###############################################################################
# Data Extraction and Vectorization

# Parameters setup
flakyZip = "Dataset/flaky_files.zip"
nonFlakyZip = "Dataset/reduced_nonflaky_files.zip"
imbalanceNonFlakyZip = "Dataset/imbalance_nonflaky_files.zip"

# Create directories
outDirbalance = "results/balance_flaky_nonflaky/"
outDirimbalance = "results/imbalance_nonflaky/"
os.makedirs(outDirbalance, exist_ok=True)
os.makedirs(outDirimbalance, exist_ok=True)

extractDirbalance = "extracted/balance_flaky_nonflaky/"
extractDirimbalance = "extracted/imbalance_nonflaky/"
os.makedirs(extractDirbalance, exist_ok=True)
os.makedirs(extractDirimbalance, exist_ok=True)

# Extract and read data once for balance combination
flakyDirbalance = os.path.join(extractDirbalance, 'flaky')
nonFlakyDirbalance = os.path.join(extractDirbalance, 'nonFlaky')
os.makedirs(flakyDirbalance, exist_ok=True)
os.makedirs(nonFlakyDirbalance, exist_ok=True)

extract_zip(flakyZip, flakyDirbalance)
extract_zip(nonFlakyZip, nonFlakyDirbalance)

dataPointsFlakybalance = getDataPoints(flakyDirbalance)
dataPointsNonFlakybalance = getDataPoints(nonFlakyDirbalance)
dataPointsbalance = dataPointsFlakybalance + dataPointsNonFlakybalance

# Print the number of datasets for balance combination
print(f"Number of flaky documents (balance combination): {len(dataPointsFlakybalance)}")
print(f"Number of non-flaky documents (balance combination): {len(dataPointsNonFlakybalance)}")
print(f"Total number of documents (balance combination): {len(dataPointsbalance)}")

dataLabelsListbalance = np.array([1]*len(dataPointsFlakybalance) + [0]*len(dataPointsNonFlakybalance))

# Vectorize data once
Z_balance = flastVectorization(dataPointsbalance, dim=100, eps=0.3)

# Extract and read data once for imbalance non-flaky combination
flakyDirimbalance = os.path.join(extractDirimbalance, 'flaky')
nonFlakyDirimbalance = os.path.join(extractDirimbalance, 'nonFlaky')
os.makedirs(flakyDirimbalance, exist_ok=True)
os.makedirs(nonFlakyDirimbalance, exist_ok=True)

extract_zip(flakyZip, flakyDirimbalance)
extract_zip(imbalanceNonFlakyZip, nonFlakyDirimbalance)

dataPointsFlakyimbalance = getDataPoints(flakyDirimbalance)
dataPointsNonFlakyimbalance = getDataPoints(nonFlakyDirimbalance)
dataPointsimbalance = dataPointsFlakyimbalance + dataPointsNonFlakyimbalance

# Print the number of datasets for imbalance combination
print(f"Number of flaky documents (imbalance combination): {len(dataPointsFlakyimbalance)}")
print(f"Number of non-flaky documents (imbalance combination): {len(dataPointsNonFlakyimbalance)}")
print(f"Total number of documents (imbalance combination): {len(dataPointsimbalance)}")

dataLabelsListimbalance = np.array([1]*len(dataPointsFlakyimbalance) + [0]*len(dataPointsNonFlakyimbalance))

Z_imbalance = flastVectorization(dataPointsimbalance, dim=100, eps=0.3)


Number of flaky documents (balance combination): 45
Number of non-flaky documents (balance combination): 45
Total number of documents (balance combination): 90
Number of flaky documents (imbalance combination): 45
Number of non-flaky documents (imbalance combination): 243
Total number of documents (imbalance combination): 288


## RF

In [ ]:
import time
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
from sklearn.model_selection import StratifiedKFold, GridSearchCV

###############################################################################
# Random Forest with GridSearchCV and Multiple Scoring Metrics including MCC



def runRandomForest(Z, dataLabelsList, outDir, n_splits, combination_label):
    v0 = time.perf_counter()
    
    # Define Random Forest model
    rf_model = RandomForestClassifier()
    
    # Define parameter grid for hyperparameter tuning
    param_grid = {
        'n_estimators': [50, 100, 200],  # Number of trees
        'max_depth': [10, 20,30,50],  # Maximum depth of each tree
        'min_samples_split': [5, 10],  # Minimum number of samples required to split an internal node
        'min_samples_leaf': [2, 5],  # Minimum number of samples required to be at a leaf node
        "criterion": ["gini", "entropy"],  # Function to measure the quality of a split
    }

    # Custom scoring functions including MCC
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),  
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': make_scorer(matthews_corrcoef)
    }

    # Cross-validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Perform GridSearchCV with multiple scoring metrics
    grid_search = GridSearchCV(rf_model, param_grid, cv=skf, scoring=scoring, refit='f1', verbose=1, return_train_score=True)

    # Fit the GridSearchCV on training data
    grid_search.fit(Z, dataLabelsList)

    # Get the best parameters and the best score for F1
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    print(f"Best Parameters: {best_params}")
    print(f"Best F1 Score: {best_score}")

    # Save the results
    outFile = f"{combination_label}-params-rf-{n_splits}-folds.csv"
    with open(os.path.join(outDir, outFile), "w") as fo:
        fo.write("n_estimators,max_depth,min_samples_split,min_samples_leaf,criterion,accuracy,precision,recall,f1,mcc,preparationTime\n")
        for idx, param in enumerate(grid_search.cv_results_['params']):
            accuracy = grid_search.cv_results_['mean_test_accuracy'][idx]
            precision = grid_search.cv_results_['mean_test_precision'][idx]
            recall = grid_search.cv_results_['mean_test_recall'][idx]
            f1 = grid_search.cv_results_['mean_test_f1'][idx]
            mcc = grid_search.cv_results_['mean_test_mcc'][idx]
            preparationTime = (time.perf_counter() - v0) / len(dataLabelsList)  
            fo.write(f"{param['n_estimators']},{param['max_depth']},{param['min_samples_split']},{param['min_samples_leaf']},{param['criterion']},{accuracy},{precision},{recall},{f1},{mcc},{preparationTime}\n")

    print(f"Random Forest analysis completed for {n_splits}-folds. Results saved to: {outFile}")
    return best_params, best_score

# Run Random Forest on balance combination
print("\nStarting Random Forest analysis for flaky vs smaller non-flaky files (balance combination)...")
best_params_5folds_balance, best_score_5folds_balance = runRandomForest(Z_balance, dataLabelsListbalance, outDirbalance, 5, "balance")

print("\nBest results for Random Forest 5-fold on balance combination:")
print(f"Best Parameters: {best_params_5folds_balance}")
print(f"Best F1 Score: {best_score_5folds_balance}")


# Run Random Forest on imbalance non-flaky combination
print("\nStarting Random Forest analysis for flaky vs imbalance non-flaky files (imbalance combination)...")
best_params_5folds_imbalance, best_score_5folds_imbalance = runRandomForest(Z_imbalance, dataLabelsListimbalance, outDirimbalance, 5, "imbalance")

print("\nBest results for Random Forest 5-fold on imbalance combination:")
print(f"Best Parameters: {best_params_5folds_imbalance}")
print(f"Best F1 Score: {best_score_5folds_imbalance}")



Starting Random Forest analysis for flaky vs smaller non-flaky files (balance combination)...
Fitting 5 folds for each of 96 candidates, totalling 480 fits
Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}
Best F1 Score: 0.7720743034055728
Random Forest analysis completed for 5-folds. Results saved to: balance-params-rf-5-folds.csv

Best results for Random Forest 5-fold on balance combination:
Best Parameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}
Best F1 Score: 0.7720743034055728

Starting Random Forest analysis for flaky vs imbalance non-flaky files (imbalance combination)...
Fitting 5 folds for each of 96 candidates, totalling 480 fits
Best Parameters: {'criterion': 'entropy', 'max_depth': 30, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Best F1 Score: 0.38461538461538464
Random Forest analysis completed for 5-

## Decision Tree

In [3]:
import time
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, precision_score, recall_score, accuracy_score, f1_score, matthews_corrcoef
from sklearn.model_selection import StratifiedKFold, GridSearchCV

###############################################################################
# Decision Tree with GridSearchCV and Multiple Scoring Metrics including MCC

def mcc_scorer(estimator, X, y_true):
    """
    Custom scorer function for Matthews Correlation Coefficient.
    """
    y_pred = estimator.predict(X)
    return matthews_corrcoef(y_true, y_pred)

def runDecisionTree(Z, dataLabelsList, outDir, n_splits, combination_label):
    v0 = time.perf_counter()

    # Define Decision Tree model
    dt_model = DecisionTreeClassifier()

    # Define parameter grid for hyperparameter tuning
    param_grid = {
        'criterion': ['gini', 'entropy'],
        'max_depth': [ 10,20,30],
        'min_samples_split': [2, 5, 10],  
        'min_samples_leaf': [1, 2, 5, 10],  
        'max_features': [None, 'sqrt', 'log2'], 
    }

    # Custom scoring functions including MCC
    scoring = {
        'precision': make_scorer(precision_score, zero_division=1),  
        'recall': make_scorer(recall_score, zero_division=1),
        'accuracy': make_scorer(accuracy_score),
        'f1': make_scorer(f1_score, zero_division=1),
        'mcc': mcc_scorer  # Use the custom MCC scorer
    }

    # Cross-validation
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Perform GridSearchCV with multiple scoring metrics
    grid_search = GridSearchCV(
        dt_model, 
        param_grid, 
        cv=skf, 
        scoring=scoring, 
        refit='f1', 
        verbose=1, 
        return_train_score=True
    )

    # Fit the GridSearchCV on training data
    grid_search.fit(Z, dataLabelsList)

    # Get the best parameters and the best score for f1
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    print(f"Best Parameters: {best_params}")
    print(f"Best F1 Score: {best_score}")

    # Save the results
    outFile = f"{combination_label}-params-dt-{n_splits}-folds.csv"
    with open(os.path.join(outDir, outFile), "w") as fo:
        fo.write("criterion,max_depth,min_samples_split,min_samples_leaf,max_features,accuracy,precision,recall,f1,mcc,preparationTime\n")
        for idx, param in enumerate(grid_search.cv_results_['params']):
            accuracy = grid_search.cv_results_['mean_test_accuracy'][idx]
            precision = grid_search.cv_results_['mean_test_precision'][idx]
            recall = grid_search.cv_results_['mean_test_recall'][idx]
            f1 = grid_search.cv_results_['mean_test_f1'][idx]
            mcc = grid_search.cv_results_['mean_test_mcc'][idx]
            preparationTime = (time.perf_counter() - v0) / len(dataLabelsList)  
            fo.write(f"{param['criterion']},{param['max_depth']},{param['min_samples_split']},{param['min_samples_leaf']},{param['max_features']},{accuracy},{precision},{recall},{f1},{mcc},{preparationTime}\n")

    print(f"Decision Tree analysis completed for {n_splits}-folds. Results saved to: {outFile}")
    return best_params, best_score

# Run Decision Tree on balance combination
print("\nStarting Decision Tree analysis for flaky vs smaller non-flaky files (balance combination)...")
best_params_5folds_balance, best_score_5folds_balance = runDecisionTree(
    Z_balance, dataLabelsListbalance, outDirbalance, 5, "balance"
)

print("\nBest results for Decision Tree 5-fold on balance combination:")
print(f"Best Parameters: {best_params_5folds_balance}")
print(f"Best F1 Score: {best_score_5folds_balance}")

# Run Decision Tree on imbalance non-flaky combination
print("\nStarting Decision Tree analysis for flaky vs imbalance non-flaky files (imbalance combination)...")
best_params_5folds_imbalance, best_score_5folds_imbalance = runDecisionTree(
    Z_imbalance, dataLabelsListimbalance, outDirimbalance, 5, "imbalance"
)

print("\nBest results for Decision Tree 5-fold on imbalance combination:")
print(f"Best Parameters: {best_params_5folds_imbalance}")
print(f"Best F1 Score: {best_score_5folds_imbalance}")



Starting Decision Tree analysis for flaky vs smaller non-flaky files (balance combination)...
Fitting 5 folds for each of 216 candidates, totalling 1080 fits


c:\Users\haha9\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Parameters: {'criterion': 'gini', 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 10}
Best F1 Score: 0.7907953484424072
Decision Tree analysis completed for 5-folds. Results saved to: balance-params-dt-5-folds.csv

Best results for Decision Tree 5-fold on balance combination:
Best Parameters: {'criterion': 'gini', 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 10}
Best F1 Score: 0.7907953484424072

Starting Decision Tree analysis for flaky vs imbalance non-flaky files (imbalance combination)...
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best Parameters: {'criterion': 'gini', 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5}
Best F1 Score: 0.6054621848739495
Decision Tree analysis completed for 5-folds. Results saved to: imbalance-params-dt-5-folds.csv

Best results for Decision Tree 5-fold on imbalance combination:
Best Parameters: {'criterion': 'gi